In [ ]:
# First check the Python version
import sys
if sys.version_info < (3,4):
    print('You are running an older version of Python!\n\n',
          'You should consider updating to Python 3.4.0 or',
          'higher as the libraries built for this course',
          'have only been tested in Python 3.4 and higher.\n')
    print('Try installing the Python 3.5 version of anaconda'
          'and then restart `jupyter notebook`:\n',
          'https://www.continuum.io/downloads\n\n')

# Now get necessary libraries
try:
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    from skimage.transform import resize
    from skimage import data
    from scipy.misc import imresize
    from scipy.ndimage.filters import gaussian_filter
    import IPython.display as ipyd
    import tensorflow as tf
    from libs import utils, gif, datasets, dataset_utils, vae, dft, vgg16, nb_utils
except ImportError:
    print("Make sure you have started notebook in the same directory",
          "as the provided zip file which includes the 'libs' folder",
          "and the file 'utils.py' inside of it.  You will NOT be able",
          "to complete this assignment unless you restart jupyter",
          "notebook inside the directory created by extracting",
          "the zip file or cloning the github repo.  If you are still")

# We'll tell matplotlib to inline any drawn figures like so:
%matplotlib inline
plt.style.use('ggplot')

# Bit of formatting because I don't like the default inline code style:
from IPython.core.display import HTML
HTML("""<style> .rendered_html code { 
    padding: 2px 4px;
    color: #c7254e;
    background-color: #f9f2f4;
    border-radius: 4px;
} </style>""")



In [ ]:
from libs import vgg16, inception, i2v

In [ ]:
net = inception.get_inception_model(version='v5')

In [ ]:
og = plt.imread('raj1.jpg')[..., :3]
plt.imshow(og)

In [ ]:
og = plt.imread('raj1.jpg')[..., :3]
plt.imshow(og)


In [ ]:
deprocessed = net['preprocess'](img)
plt.imshow(deprocessed)
plt.show()

In [ ]:
net = vgg16.get_vgg_model()

In [ ]:

device = '/cpu:0'

g = tf.Graph()


with tf.Session(graph=g) as sess, g.device(device):
    
    tf.import_graph_def(net['graph_def'], name='net')

In [ ]:
features = ['net/pool1:0', 'net/pool2:0', 'net/pool3:0']

# Let's print them
print(features)


In [ ]:
x = g.get_tensor_by_name('net/images:0');

assert(x.name == 'net/images:0')

In [ ]:
def plot_gradient(img, x, feature, g, device='/cpu:0'):
   
    with tf.Session(graph=g) as sess, g.device(device):
        saliency = tf.gradients(tf.reduce_mean(feature), x)
        this_res = sess.run(saliency[0], feed_dict={x: img})
        grad = this_res[0] / np.max(np.abs(this_res))
        return grad

In [ ]:
og = plt.imread('raj1.jpg')[..., :3]
img = net['preprocess'](og)[np.newaxis]

fig, axs = plt.subplots(1, len(features), figsize=(20, 10))

for i in range(len(features)):
    axs[i].set_title(features[i])
    grad = plot_gradient(img, x, g.get_tensor_by_name(features[i]), g)
    axs[i].imshow(utils.normalize(grad))

In [ ]:
def dream(img, gradient, step, net, x, n_iterations=50, plot_step=10):
    # Copy the input image as we'll add the gradient to it in a loop
    img_copy = img.copy()

    fig, axs = plt.subplots(1, n_iterations // plot_step, figsize=(20, 10))

    with tf.Session(graph=g) as sess, g.device(device):
        for it_i in range(n_iterations):

            # This will calculate the gradient of the layer we chose with respect to the input image.
            this_res = sess.run(gradient[0], feed_dict={x: img_copy})[0]

            # Let's normalize it by the maximum activation
            this_res /= (np.max(np.abs(this_res) + 1e-8))
            
            img_copy += this_res * step

            # Plot the image
            if (it_i + 1) % plot_step == 0:
                m = net['deprocess'](img_copy[0])
                axs[it_i // plot_step].imshow(m)

In [ ]:
n_iterations = 3


step = 1.0

# Every 1 iterations, we'll plot the current deep dream
plot_step = 1

In [ ]:
for feature_i in range(len(features)):
    with tf.Session(graph=g) as sess, g.device(device):
        # Get a feature layer
        layer = g.get_tensor_by_name(features[feature_i])

        gradient = tf.gradients(tf.reduce_mean(layer), x)
        
        
        dream(img, gradient, step, net, x, n_iterations=n_iterations, plot_step=plot_step)

In [ ]:
og = plt.imread('raj1.jpg')
plt.imshow(og)

# Preprocess the image and make sure it is 4-dimensional by adding a new axis to the 0th dimension:
img = net['preprocess'](og)
#xm=img.shape();
print(img.shape)
#img = img.reshape(1,244,244,3)
img = np.expand_dims(img, axis=0)
print(img.shape)
assert(img.ndim == 4)

In [ ]:
layer = g.get_tensor_by_name(names[-2] + ":0")

# And find its shape
with tf.Session(graph=g) as sess, g.device(device):
    layer_shape = tf.shape(layer).eval(feed_dict={x:img})

# We can find out how many neurons it has by feeding it an image and
# calculating the shape.  The number of output channels is the last dimension.
n_els = layer_shape[-1]

In [ ]:
neuron_i = 2 

print(net['labels'][neuron_i])
#assert(neuron_i >= 0 and neuron_i < n_els)

In [ ]:
# And we'll create an activation of this layer which is very close to 0
layer_vec = np.ones(layer_shape) / 100.0

# Except for the randomly chosen neuron which will be very close to 1
layer_vec[..., neuron_i] = 0.99

In [ ]:
# Explore different parameters for this section.
n_iterations = 51

plot_step = 5

# If you use a different network, you will definitely need to experiment
# with the step size, as each network normalizes the input image differently.
step = 0.2

In [ ]:
imgs = []
with tf.Session(graph=g) as sess, g.device(device):
    gradient = tf.gradients(tf.reduce_max(layer), x)

    # Copy the input image as we'll add the gradient to it in a loop
    img_copy = img.copy()

    with tf.Session(graph=g) as sess, g.device(device):
        for it_i in range(n_iterations):

            # This will calculate the gradient of the layer we chose with respect to the input image.
            this_res = sess.run(gradient[0], feed_dict={
                    x: img_copy, layer: layer_vec})[0]
            
            # Let's normalize it by the maximum activation
            this_res /= (np.max(np.abs(this_res) + 1e-8))
            
            # Or alternatively, we can normalize by standard deviation
            # this_res /= (np.std(this_res) + 1e-8)

            # Then add the gradient back to the input image
            # Think about what this gradient represents?
            # It says what direction we should move our input
            # in order to meet our objective stored in "gradient"
            img_copy += this_res * step

            # Plot the image
            if (it_i + 1) % plot_step == 0:
                m = net['deprocess'](img_copy[0])

                plt.figure(figsize=(5, 5))
                plt.grid('off')
                plt.imshow(m)
                plt.show()
                
                imgs.append(m)
                